In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow # cv2_imshow(img)
import cv2 as cv
import urllib.request
import numpy as np
import time
import tensorflow as tf

# for file path
import natsort
import platform
import glob
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
import PIL.Image
import matplotlib.image as img
import IPython.display as display

In [ ]:
def model_layers(layer_names): # 전역 변수, StyleContentModel에 이용
    """ 중간층의 출력값을 배열로 반환하는 < > 모델을 만듭니다."""
    # 이미지넷 데이터셋에 사전학습된 < > 모델을 불러옵니다
    # ✅ 모델 선택 가능  tf.keras.applications 문서 참고
    model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    model.trainable = False    
    outputs = [model.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([model.input], outputs, verbose=2)
    return model

In [ ]:
def load_img(path_to_img): # main, 전역변수
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img, new_shape   # 사진 크기에 따른 자동화 문제 해결

In [ ]:
def gram_matrix(input_tensor): # StyleContentModel
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [ ]:
class StyleContentModel(tf.keras.models.Model): # 전역변수 extractor
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.model =  model_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.model.trainable = False

  def call(self, inputs):
    "[0,1] 사이의 실수 값을 입력으로 받습니다"
    inputs = inputs*255.0
    # ✅ 모델 바꿀 시 변경 필요
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    #
    outputs = self.model(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}


In [ ]:
def clip_0_1(image):  # train_step
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
def style_content_loss(outputs):  # train_step
    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    # ✅ hyper parameter
    style_weight=1e-2
    content_weight=1e4
    #
    num_content_layers = len(content_layers)
    num_style_layers = len(style_layers)
    
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                        for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                            for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [ ]:
def train_step(image):  # get_transformed_img
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)
    # ✅ optimizer 선택 가능
    opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [ ]:
def tensor_to_image(tensor): # get_transformed_img
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:
def get_transformed_img(content_image): # main
    image = tf.Variable(content_image)

    # ✅ hyper parameter
    epochs = 20
    steps_per_epoch = 3
    step = 0

    for n in range(epochs):
        for m in range(steps_per_epoch):
            step += 1
            train_step(image)
            print(".", end='')
        # display.clear_output(wait=True)
        # display.display(tensor_to_image(image)
        # print("훈련 스텝: {}".format(step)) 
        
    img_transformed = np.array(tensor_to_image(image))
    return img_transformed

In [ ]:
# 전역 변수
# ✅ 모델 바꿀 시 변경 필요
content_layers = ['block5_conv2']  # style_content_loss, 전역변수  extractor
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']     # style_content_loss, 전역변수 extractor

In [ ]:
 # 작동하는지 확인하는 코드

 """
  content_location = "/content/drive/Shareddrives/SWTube/프로젝트/044. 2021 소융튜브 특성화 프로젝트/데이터 팀/mount_drive/Darkest-Cave/Data/Content_Images/goblin/Layer 1_sprite_01.png"
  
  style_image, style_shape = load_img(style_location)
  content_image, content_shpae = load_img(content_location)
  plt.figure(figsize=(8,8))

  plt.subplot(221)
  plt.imshow(style_image[0])
  plt.axis('off')

  plt.subplot(222)
  plt.imshow(content_image[0])
  plt.axis('off')

  plt.show()

  origin_image = cv.imread(content_location, cv.IMREAD_UNCHANGED)
  origin_image_shape = origin_image.shape

  # style transfer
  content_image, content_shape = load_img(content_location) 
  img_transformed = get_transformed_img(content_image)
  b, g, r = cv.split(img_transformed)
  img_transformed = cv.merge([r,g,b])

  # model이 반환한 이미지와 사이즈 동일하게 origin image 변환(자동화 완료)
  shape = content_shape.numpy()
  x, y = shape
  origin_image_resized = cv.resize(origin_image, dsize=(y,x))

  # alpha channel 추가
  final = cv.cvtColor(img_transformed, cv.COLOR_RGB2RGBA)
  final[:, :, 3] = origin_image_resized[:,:,3]

  # final image 사이즈를 original image 사이즈로 변환
  x, y = origin_image_shape[0], origin_image_shape[1]
  final = cv.resize(final, dsize=(y,x))

  cv2_imshow(final)
"""

'\n content_location = "/content/drive/Shareddrives/SWTube/프로젝트/044. 2021 소융튜브 특성화 프로젝트/데이터 팀/mount_drive/Darkest-Cave/Data/Content_Images/goblin/Layer 1_sprite_01.png"\n \n style_image, style_shape = load_img(style_location)\n content_image, content_shpae = load_img(content_location)\n plt.figure(figsize=(8,8))\n\n plt.subplot(221)\n plt.imshow(style_image[0])\n plt.axis(\'off\')\n\n plt.subplot(222)\n plt.imshow(content_image[0])\n plt.axis(\'off\')\n\n plt.show()\n\n origin_image = cv.imread(content_location, cv.IMREAD_UNCHANGED)\n origin_image_shape = origin_image.shape\n\n # style transfer\n content_image, content_shape = load_img(content_location) \n img_transformed = get_transformed_img(content_image)\n b, g, r = cv.split(img_transformed)\n img_transformed = cv.merge([r,g,b])\n\n # model이 반환한 이미지와 사이즈 동일하게 origin image 변환(자동화 완료)\n shape = content_shape.numpy()\n x, y = shape\n origin_image_resized = cv.resize(origin_image, dsize=(y,x))\n\n # alpha chann

In [ ]:
# 경로 설정 명세서
platform_path = platform.platform()

IsColab = "bionic" in platform_path
if IsColab:
  style_location = "/content/drive/Shareddrives/SWTube/프로젝트/044. 2021 소융튜브 특성화 프로젝트/데이터 팀/mount_drive/Darkest-Cave/Data/Style_Images/*"
  content_location = "/content/drive/Shareddrives/SWTube/프로젝트/044. 2021 소융튜브 특성화 프로젝트/데이터 팀/mount_drive/Darkest-Cave/Data/Content_Images/*"
  styled_location = "/content/drive/Shareddrives/SWTube/프로젝트/044. 2021 소융튜브 특성화 프로젝트/데이터 팀/mount_drive/Darkest-Cave/Data/Styled_Images/"
else:
  style_location = 'Style_Images/*'
  content_location = "Content_Images/*"
  styled_location = "Styled_Images/"

# 해당 폴더 내에 존재하는 모든 폴더 리스트 불러오기
style_folder_list = glob.glob(style_location)
style_images_list = glob.glob(style_folder_list[0] + "/*")

# Style_location
# Style_Images/TEXTURE/*.jpg or png

# style_images_list, style_image 활용
# style_image = style_images_list[index]

# Content_location
  # Content_Images/ASSET_NAME/*.png
content_folder_list = glob.glob(content_location) # Content_Images/ASSET_NAME

# content_images_list, content_image 활용
# content_images_list = glob.glob(content_folder_list[index] + "/*")
# content_image = style_images_list[index]

# Styled_location
  # Styled_Images/ASSET_NAME/*.png
# asset_name = os.path.split(content_folder_list[index])

# 폴더 생성
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# 구글 코랩에서 장시간 사용하기

### F12 개발자 모드 후 Console에 입력

```javascript
function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);
```

```javascript
function CleanCurrentOutput(){ 
    var btn = document.querySelector(".output-icon.clear_outputs_enabled.output-icon-selected[title$='현재 실행 중...'] iron-icon[command=clear-focused-or-selected-outputs]"); 
    if(btn) { console.log("30분마다 출력 지우기");
     btn.click(); 
    } 
} 
setInterval(CleanCurrentOutput,1000*60*30);
```

In [ ]:
## main code
import time
start = time.time()

# 스타일 폴더 리스트
for SF_index, SF_value in enumerate(style_folder_list):
  style_images_list = glob.glob(style_folder_list[SF_index] + "/*")
  style_folder_name = os.path.split(style_folder_list[SF_index]) # horror, ice, dark .etc

  style_images_list = natsort.natsorted(style_images_list)

  # 스타일 이미지 리스트
  for SI_index, SI_value in enumerate(style_images_list):
    style_image_name = os.path.split(style_images_list[SI_index]) # horror1, horror2, horror3, .etc
    
    # 스타일 이미지 선택
    style_image, style_shape = load_img(SI_value) # style_content_loss, 전역변수 style_outputs
    style_extractor = model_layers(style_layers)
    style_outputs = style_extractor(style_image*255) # style_content_loss, StyleContentModel
    extractor = StyleContentModel(style_layers, content_layers) # style_content_loss, train_step

    # 콘텐트 폴더 리스트
    for CF_index, CF_value in enumerate(content_folder_list):

      #콘텐트 이미지 리스트
      content_images_list = glob.glob(content_folder_list[CF_index] + "/*")
      content_images_list = natsort.natsorted(content_images_list)

      # 스타일 전이
      for CI_index, CI_value in enumerate(content_images_list):
        location = CI_value

        # 저장 경로 생성
        asset_folder = os.path.split(content_folder_list[CF_index]) # gobline, zombie, male .etc
        asset_name = os.path.split(CI_value)

        save_folder_style = styled_location + style_folder_name[1]
        save_folder_style_index = save_folder_style + "/" + style_image_name[1]
        save_folder_style_index_content = save_folder_style_index + "/" + asset_folder[1]
        
        createFolder(save_folder_style) # horror, ice, dark
        createFolder(save_folder_style_index) # horror1, horror2, horror3
        createFolder(save_folder_style_index_content) # goblin, zombie, male

        save_path = save_folder_style_index_content + "/"
        save_path = save_path + asset_name[1] + f'_{CI_index}_final.png'

        # 이미 있는 파일이면 건너뛰기
        if os.path.exists(save_path):
          continue
        else:
          origin_image = cv.imread(location, cv.IMREAD_UNCHANGED)
          origin_image_shape = origin_image.shape

          # style transfer
          content_image, content_shape = load_img(location) 
          img_transformed = get_transformed_img(content_image)
          b, g, r = cv.split(img_transformed)
          img_transformed = cv.merge([r,g,b])

          # model이 반환한 이미지와 사이즈 동일하게 origin image 변환(자동화 완료)
          shape = content_shape.numpy()
          x, y = shape
          origin_image_resized = cv.resize(origin_image, dsize=(y,x))

          # alpha channel 추가
          final = cv.cvtColor(img_transformed, cv.COLOR_RGB2RGBA)
          final[:, :, 3] = origin_image_resized[:,:,3]

          # final image 사이즈를 original image 사이즈로 변환
          x, y = origin_image_shape[0], origin_image_shape[1]
          final = cv.resize(final, dsize=(y,x))

          # 저장
          cv.imwrite(save_path, final)
          cv2_imshow(final)

end = time.time()
print("전체 소요 시간: {:.1f}".format(end-start))

Output hidden; open in https://colab.research.google.com to view.